In [2]:
# Step 0

import csv
import json 
def carica_dati_netflix(path):
    dataset = []

    with open(path, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)

        for row in reader:
            record = {
                'show_id': row.get('show_id', '').strip(),
                'type': row.get('type', '').strip(),
                'title': row.get('title', '').strip(),
                'director': row.get('director', '').strip(),
                'cast': [c.strip() for c in row.get('cast', '').split(',')] if row.get('cast') else [],
                'country': row.get('country', '').strip(),
                'date_added': row.get('date_added', '').strip(),
                'release_year': int(row['release_year']) if row.get('release_year', '').isdigit() else None,
                'rating': row.get('rating', '').strip(),
                'duration': row.get('duration', '').strip(),
                'listed_in': [g.strip() for g in row.get('listed_in', '').split(',')] if row.get('listed_in') else [],
                'description': row.get('description', '').strip()
            }

            dataset.append(record)

    return dataset

# Esempio d'uso
dati = carica_dati_netflix('data/netflix_titles.csv')
print(json.dumps(dati[7659], indent=2))


{
  "show_id": "s7660",
  "type": "Movie",
  "title": "Once Upon a Time in the West",
  "director": "Sergio Leone",
  "cast": [
    "Henry Fonda",
    "Charles Bronson",
    "Claudia Cardinale",
    "Jason Robards",
    "Gabriele Ferzetti",
    "Paolo Stoppa",
    "Woody Strode",
    "Jack Elam",
    "Keenan Wynn",
    "Frank Wolff",
    "Lionel Stander"
  ],
  "country": "Italy, United States",
  "date_added": "November 20, 2019",
  "release_year": 1968,
  "rating": "PG-13",
  "duration": "166 min",
  "listed_in": [
    "Action & Adventure",
    "Classic Movies",
    "International Movies"
  ],
  "description": "In this epic spaghetti Western, a flinty gunslinger is hired by a railroad tycoon to kill anyone standing in the way of his trans-American iron horse."
}


In [2]:
# Step 1

def raccomanda_per_genere(dati, genere_input):
    genere_input = genere_input.lower()
    risultati = []

    for record in dati:
        generi = [g.lower() for g in record['listed_in']]
        if genere_input in generi:
            risultati.append(record)

    risultati_ordinati = sorted(risultati, key=lambda r: r['release_year'] or 0, reverse=True)

    for r in risultati_ordinati:
        print(f"{r['title']} ({r['release_year']}) — {', '.join(r['listed_in'])}")

def raccomanda_per_generi(dati, input_generi):
    # Pulisci e dividi i generi richiesti
    generi_richiesti = [g.strip().lower() for g in input_generi.split(',')]

    risultati = []

    for record in dati:
        generi_film = [g.lower() for g in record['listed_in']]
        
        # Verifica se almeno un genere combacia
        if any(gen in generi_film for gen in generi_richiesti):
            risultati.append(record)

    # Ordina per anno decrescente
    risultati_ordinati = sorted(risultati, key=lambda r: r['release_year'] or 0, reverse=True)

    # Stampa risultati
    for r in risultati_ordinati:
        print(f"{r['title']} ({r['release_year']}) — {', '.join(r['listed_in'])}")

# ESEMPIO USO
generi = input("Inserisci uno o più generi separati da virgola (es. Drama, Thriller): ")
raccomanda_per_generi(dati, generi)


# ESEMPIO D'USO
genere = input("Inserisci un genere (es. Dramas, Thrillers): ")
raccomanda_per_genere(dati, genere)


Inserisci uno o più generi separati da virgola (es. Drama, Thriller):  once upon a time in the west
Inserisci un genere (es. Dramas, Thrillers):  dramas


The Starling (2021) — Comedies, Dramas
Je Suis Karl (2021) — Dramas, International Movies
Ankahi Kahaniya (2021) — Dramas, Independent Movies, International Movies
The Father Who Moves Mountains (2021) — Dramas, International Movies, Thrillers
The Stronghold (2021) — Action & Adventure, Dramas, International Movies
Tughlaq Durbar (Telugu) (2021) — Comedies, Dramas, International Movies
JJ+E (2021) — Dramas, International Movies, Romantic Movies
Worth (2021) — Dramas
Thimmarusu (2021) — Dramas, International Movies
The Water Man (2021) — Children & Family Movies, Dramas
Man in Love (2021) — Dramas, International Movies, Romantic Movies
Sweet Girl (2021) — Action & Adventure, Dramas
I missed you: Director's Cut (2021) — Dramas, Independent Movies, International Movies
Mimi (2021) — Comedies, Dramas, International Movies
African America (2021) — Dramas, Independent Movies, International Movies
The Last Letter From Your Lover (2021) — Dramas, Romantic Movies
Cousins (2021) — Dramas
The Tam

In [3]:
data

NameError: name 'data' is not defined

Concetti introdotti: 
- Feature Space
- One Hot Encoding di tipo binario
- Similarità del coseno
- Concetto di "prossimità"

In [3]:
import math

# Crea un vocabolario ordinato di tutti i generi presenti nel dataset (senza duplicati)
def costruisci_vocabolario_generi(dati):
    generi = set()
    for record in dati:
        for g in record['listed_in']:
            generi.add(g.strip().lower())
    return sorted(list(generi))

# Converte i generi di un film in un vettore binario in base al vocabolario
def vettore_generi(record, vocabolario):
    generi_film = set(g.lower() for g in record['listed_in'])
    return [1 if g in generi_film else 0 for g in vocabolario]

# Calcola la similarità coseno tra due vettori (quanto sono orientati nella stessa direzione)
def similarita_coseno(v1, v2):
    dot = sum(a*b for a, b in zip(v1, v2))
    norm1 = math.sqrt(sum(a*a for a in v1))
    norm2 = math.sqrt(sum(b*b for b in v2))
    if norm1 == 0 or norm2 == 0:
        return 0
    return dot / (norm1 * norm2)

# Cerca un film nel dataset confrontando il titolo in modo case-insensitive
def trova_record_per_titolo(dati, titolo_input):
    titolo_input = titolo_input.strip().lower()
    for record in dati:
        if record['title'].strip().lower() == titolo_input:
            return record
    return None

# Mostra i top-N titoli più simili a quello scelto, basandosi sui generi
def raccomanda_simili(dati, titolo_input, top_n=10):
    vocabolario = costruisci_vocabolario_generi(dati)
    film_richiesto = trova_record_per_titolo(dati, titolo_input)

    if film_richiesto is None:
        print("Titolo non trovato.")
        return

    vettore_input = vettore_generi(film_richiesto, vocabolario)
    risultati = []

    for record in dati:
        if record['title'] == film_richiesto['title']:
            continue
        v = vettore_generi(record, vocabolario)
        sim = similarita_coseno(vettore_input, v)
        risultati.append((record['title'], sim, record['listed_in']))

    top = sorted(risultati, key=lambda x: x[1], reverse=True)[:top_n]

    print(f"\nTitoli più simili a: {film_richiesto['title']} {film_richiesto['listed_in']}\n")
    for titolo, score, genres in top:
        print(f"{titolo} — similarità: {round(score, 2)} -> {genres}\n")

print("GENERI: ", costruisci_vocabolario_generi(dati), "\n")
print(dati[7659]['title'], '->', dati[7659]['listed_in'])
print("VETTORE:", vettore_generi(dati[7659], costruisci_vocabolario_generi(dati)), "\n")
print("Cos Similarity: ", similarita_coseno([0, 1, 0], [1, 0, 0]), "\n")

GENERI:  ['action & adventure', 'anime features', 'anime series', 'british tv shows', 'children & family movies', 'classic & cult tv', 'classic movies', 'comedies', 'crime tv shows', 'cult movies', 'documentaries', 'docuseries', 'dramas', 'faith & spirituality', 'horror movies', 'independent movies', 'international movies', 'international tv shows', "kids' tv", 'korean tv shows', 'lgbtq movies', 'movies', 'music & musicals', 'reality tv', 'romantic movies', 'romantic tv shows', 'sci-fi & fantasy', 'science & nature tv', 'spanish-language tv shows', 'sports movies', 'stand-up comedy', 'stand-up comedy & talk shows', 'teen tv shows', 'thrillers', 'tv action & adventure', 'tv comedies', 'tv dramas', 'tv horror', 'tv mysteries', 'tv sci-fi & fantasy', 'tv shows', 'tv thrillers'] 

Once Upon a Time in the West -> ['Action & Adventure', 'Classic Movies', 'International Movies']
VETTORE: [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [4]:
titolo = input("Inserisci il titolo di un film o serie: ")
raccomanda_simili(dati, titolo)

Inserisci il titolo di un film o serie:  once upon a time in the west



Titoli più simili a: Once Upon a Time in the West ['Action & Adventure', 'Classic Movies', 'International Movies']

Initial D — similarità: 0.82 -> ['Action & Adventure', 'International Movies']

The Guns of Navarone — similarità: 0.82 -> ['Action & Adventure', 'Classic Movies']

In the Line of Fire — similarità: 0.82 -> ['Action & Adventure', 'Classic Movies']

SAS: Rise of the Black Swan — similarità: 0.82 -> ['Action & Adventure', 'International Movies']

Rurouni Kenshin: The Beginning — similarità: 0.82 -> ['Action & Adventure', 'International Movies']

Bartkowiak — similarità: 0.82 -> ['Action & Adventure', 'International Movies']

Department — similarità: 0.82 -> ['Action & Adventure', 'International Movies']

Brick Mansions — similarità: 0.82 -> ['Action & Adventure', 'International Movies']

Major Grom: Plague Doctor — similarità: 0.82 -> ['Action & Adventure', 'International Movies']

Dynasty Warriors — similarità: 0.82 -> ['Action & Adventure', 'International Movies']



Adesso andiamo a vettorizzare anche altre colonne e non solo i generi: 

- rating
- type
- director (solo gli N più frequenti)

## Uso di Counter

```python
from collections import Counter

nomi = ['Luca', 'Anna', 'Luca', 'Marco', 'Anna', 'Luca']
conteggio = Counter(nomi)
print(conteggio)

# output
Counter({'Luca': 3, 'Anna': 2, 'Marco': 1})

```

Versione senza Counter

```python
director_counter = {}

for r in dati:
    if r['director']:
        nome = r['director'].strip()
        if nome not in director_counter:
            director_counter[nome] = 1
        else:
            director_counter[nome] += 1

```

In [34]:
from collections import Counter

# Restituisce i vocabolari unici ordinati per rating, tipo, e i top N registi più frequenti
def costruisci_vocabolari_categorici(dati, top_n_directors=20):
    ratings = set()
    types = set()
    director_counter = Counter()

    for r in dati:
        # Questi if servono ad evitare di avere valori vuoti nei vari set
        if r['rating']:
            ratings.add(r['rating'].strip())
        if r['type']:
            types.add(r['type'].strip())
        if r['director']:
            nomi = [n.strip() for n in r['director'].split(',')]
            for nome in nomi:
                director_counter[nome] += 1

    top_directors = [d for d, _ in director_counter.most_common(top_n_directors)]

    return sorted(list(ratings)), sorted(list(types)), top_directors


def vettore_completo(record, voc_generi, voc_rating, voc_type, voc_directors):
    v_gen = vettore_generi(record, voc_generi)

    v_rat = [1 if record['rating'] == r else 0 for r in voc_rating]
    v_typ = [1 if record['type'] == t else 0 for t in voc_type]
    v_dir = [1 if d in record['director'] else 0 for d in voc_directors]

    return v_gen + v_rat + v_typ + v_dir

def raccomanda_simili_esteso(dati, titolo_input, top_n=5):
    voc_gen = costruisci_vocabolario_generi(dati)
    voc_rat, voc_typ, voc_dir = costruisci_vocabolari_categorici(dati)

    film_richiesto = trova_record_per_titolo(dati, titolo_input)
    if film_richiesto is None:
        print("Titolo non trovato.")
        return

    v_input = vettore_completo(film_richiesto, voc_gen, voc_rat, voc_typ, voc_dir)

    risultati = []
    for r in dati:
        if r['title'] == film_richiesto['title']:
            continue
        v_altro = vettore_completo(r, voc_gen, voc_rat, voc_typ, voc_dir)
        sim = similarita_coseno(v_input, v_altro)
        risultati.append((r['title'], sim))

    top = sorted(risultati, key=lambda x: x[1], reverse=True)[:top_n]

    print(f"\nTitoli più simili a: {film_richiesto['title']}\n")
    for titolo, score in top:
        print(f"{titolo} — similarità: {round(score, 2)}\n")

top_n_directors = 20

ratings, types, directors = costruisci_vocabolari_categorici(dati, top_n_directors)

print("RATING: ", ratings, "\n")
print("TYPES: ", types, "\n")
print(f"TOP {top_n_directors} DIRECTORS: ", directors, "\n")


once = dati[7659]

print(f"{once['title']}, girato da {once['director']}; generi: {once['listed_in']}" )
generi = costruisci_vocabolario_generi(dati)
vettore = vettore_completo(
    once, 
    generi,
    ratings,
    types,
    directors
)

print(vettore)

RATING:  ['66 min', '74 min', '84 min', 'G', 'NC-17', 'NR', 'PG', 'PG-13', 'R', 'TV-14', 'TV-G', 'TV-MA', 'TV-PG', 'TV-Y', 'TV-Y7', 'TV-Y7-FV', 'UR'] 

TYPES:  ['Movie', 'TV Show'] 

TOP 20 DIRECTORS:  ['Rajiv Chilaka', 'Jan Suter', 'Raúl Campos', 'Suhas Kadav', 'Marcus Raboy', 'Jay Karas', 'Cathy Garcia-Molina', 'Youssef Chahine', 'Martin Scorsese', 'Jay Chapman', 'Steven Spielberg', 'Don Michael Paul', 'David Dhawan', 'Yılmaz Erdoğan', 'Anurag Kashyap', 'Shannon Hartman', 'Quentin Tarantino', 'Robert Rodriguez', 'Hakan Algül', 'Hanung Bramantyo'] 

Once Upon a Time in the West, girato da Sergio Leone; generi: ['Action & Adventure', 'Classic Movies', 'International Movies']
[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [35]:
titolo = input("Inserisci il titolo di un film o serie: ")
raccomanda_simili_esteso(dati, titolo)

Inserisci il titolo di un film o serie:  once upon a time in the west



Titoli più simili a: Once Upon a Time in the West

Brick Mansions — similarità: 0.89

Dances with Wolves — similarità: 0.8

Stardust — similarità: 0.8

The Grandmaster — similarità: 0.8

Lupin the 3rd: The Castle of Cagliostro: Special Edition — similarità: 0.8



## TF/IDF per le descrizioni



In [22]:
def tokenizza_testo(testo):
    testo = testo.lower()
    simboli = '.,!?()[]{}:;\"\''
    for s in simboli:
        testo = testo.replace(s, '')
    parole = testo.split()
    return parole

def calcola_tf(dataset):
    tf_per_film = []

    for record in dataset:
        parole = tokenizza_testo(record['description']) if record['description'] else []
        totale = len(parole)
        tf = {}
        for p in parole:
            tf[p] = tf.get(p, 0) + 1
        if totale > 0:
            for p in tf:
                tf[p] /= totale
        tf_per_film.append(tf)

    return tf_per_film

def calcola_idf(tf_per_film):
    df = {}
    N = len(tf_per_film)

    for tf in tf_per_film:
        for parola in tf.keys():
            df[parola] = df.get(parola, 0) + 1

    idf = {}
    for parola, conteggio in df.items():
        idf[parola] = math.log(N / (1 + conteggio))

    return idf

def calcola_tfidf(tf_per_film, idf):
    vettori = []

    for tf in tf_per_film:
        vettore = {}
        for parola in tf:
            vettore[parola] = tf[parola] * idf.get(parola, 0)
        vettori.append(vettore)

    return vettori

def similarita_coseno_sparsa(v1, v2):
    parole_comuni = set(v1.keys()) & set(v2.keys())
    numeratore = sum(v1[p] * v2[p] for p in parole_comuni)

    norm1 = math.sqrt(sum(v**2 for v in v1.values()))
    norm2 = math.sqrt(sum(v**2 for v in v2.values()))

    if norm1 == 0 or norm2 == 0:
        return 0.0

    return numeratore / (norm1 * norm2)


def raccomanda_simili_descrizione(dataset, vettori_tfidf, titolo_input, top_n=5):
    titolo_input = titolo_input.strip().lower()
    index = next((i for i, r in enumerate(dataset) if r['title'].strip().lower() == titolo_input), None)

    if index is None:
        print("Titolo non trovato.")
        return

    v_input = vettori_tfidf[index]
    risultati = []

    for i, v in enumerate(vettori_tfidf):
        if i == index:
            continue
        sim = similarita_coseno_sparsa(v_input, v)
        risultati.append((dataset[i]['title'], sim, dataset[i]['description']))

    top = sorted(risultati, key=lambda x: x[1], reverse=True)[:top_n]

    print(f"\nTitoli più simili a '{dataset[index]['title']}' per descrizione:\n")
    for titolo, score, description in top:
        print(f"{titolo} — similarità: {round(score, 2)}\n{description}\n\n")

In [24]:
tf = calcola_tf(dati)
idf = calcola_idf(tf)
tfidf = calcola_tfidf(tf, idf)

print(dati[7659]['description'])
print(json.dumps(tf[7659], indent=4))
print(f"Term Frequency for 'his' -> {tf[7659]['his']}")

In this epic spaghetti Western, a flinty gunslinger is hired by a railroad tycoon to kill anyone standing in the way of his trans-American iron horse.
{
    "in": 0.07692307692307693,
    "this": 0.038461538461538464,
    "epic": 0.038461538461538464,
    "spaghetti": 0.038461538461538464,
    "western": 0.038461538461538464,
    "a": 0.07692307692307693,
    "flinty": 0.038461538461538464,
    "gunslinger": 0.038461538461538464,
    "is": 0.038461538461538464,
    "hired": 0.038461538461538464,
    "by": 0.038461538461538464,
    "railroad": 0.038461538461538464,
    "tycoon": 0.038461538461538464,
    "to": 0.038461538461538464,
    "kill": 0.038461538461538464,
    "anyone": 0.038461538461538464,
    "standing": 0.038461538461538464,
    "the": 0.038461538461538464,
    "way": 0.038461538461538464,
    "of": 0.038461538461538464,
    "his": 0.038461538461538464,
    "trans-american": 0.038461538461538464,
    "iron": 0.038461538461538464,
    "horse": 0.038461538461538464
}
Term Fre

In [26]:
print(idf['in'])

0.8704625541289751


In [30]:
tfidf

[{'as': 0.07517468191154751,
  'her': 0.06514119109099731,
  'father': 0.1340544568066611,
  'nears': 0.2643478514722276,
  'the': 0.0364667422403335,
  'end': 0.1907954261511531,
  'of': 0.0277611530315511,
  'his': 0.09315980789522,
  'life': 0.09044888774314326,
  'filmmaker': 0.1907954261511531,
  'kirsten': 0.310746479935019,
  'johnson': 0.27680978616190216,
  'stages': 0.23867573367371112,
  'death': 0.1445006884634583,
  'in': 0.032239353856628704,
  'inventive': 0.27005713591027425,
  'and': 0.02090778614909352,
  'comical': 0.27005713591027425,
  'ways': 0.18733149807667815,
  'to': 0.021768194352456884,
  'help': 0.12616845690878434,
  'them': 0.12395295708615806,
  'both': 0.1685445424315631,
  'face': 0.16335086312383532,
  'inevitable': 0.2957292537087907},
 {'after': 0.08091065201761005,
  'crossing': 0.27680978616190216,
  'paths': 0.1879988501323329,
  'at': 0.18620894210304914,
  'a': 0.03644859727675284,
  'party': 0.1818116618114749,
  'cape': 0.27680978616190216,
 

In [29]:
titolo = input("Inserisci un titolo: ")
print(once['description'])
raccomanda_simili_descrizione(dati, tfidf, titolo)

Inserisci un titolo:  once upon a time in the west


In this epic spaghetti Western, a flinty gunslinger is hired by a railroad tycoon to kill anyone standing in the way of his trans-American iron horse.

Titoli più simili a 'Once Upon a Time in the West' per descrizione:

Young & Hungry — similarità: 0.14
A female food blogger is hired as a personal chef by a young tech tycoon and faces a new menu of challenges, both inside and outside of the kitchen.


One-Punch Man — similarità: 0.13
The most powerful superhero in the world can kill anyone with one blow. But nothing can challenge him, so he struggles with ennui and depression.


Spirit: Riding Free — similarità: 0.11
In a small Western town, spunky ex-city girl Lucky forms a tight bond with wild horse Spirit while having adventures with best pals Pru and Abigail.


Railroad Tigers — similarità: 0.11
During World War II, a Chinese railroad worker leads a group of brave resistance fighters against the forces of Japanese occupation.


Under Siege — similarità: 0.11
A disgruntled former C

In [42]:
def similarita_pesata(v_tfidf1, v_struct1, v_tfidf2, v_struct2, peso_tfidf=0.7):
    sim_tfidf = similarita_coseno_sparsa(v_tfidf1, v_tfidf2)
    sim_struct = similarita_coseno(v_struct1, v_struct2)
    return peso_tfidf * sim_tfidf + (1 - peso_tfidf) * sim_struct


def raccomanda_ibrido(dati, vettori_tfidf, titolo_input, peso_tfidf=0.7, top_n=5):
    voc_gen = costruisci_vocabolario_generi(dati)
    voc_rat, voc_typ, voc_dir = costruisci_vocabolari_categorici(dati)

    titolo_input = titolo_input.strip().lower()
    index = next((i for i, r in enumerate(dati) if r['title'].strip().lower() == titolo_input), None)

    if index is None:
        print("Titolo non trovato.")
        return

    v_tfidf_input = vettori_tfidf[index]
    v_struct_input = vettore_completo(dati[index], voc_gen, voc_rat, voc_typ, voc_dir)

    risultati = []
    for i, r in enumerate(dati):
        if i == index:
            continue

        v_tfidf = vettori_tfidf[i]
        v_struct = vettore_completo(r, voc_gen, voc_rat, voc_typ, voc_dir)

        sim = similarita_pesata(v_tfidf_input, v_struct_input, v_tfidf, v_struct, peso_tfidf)
        risultati.append((r['title'], sim))

    top = sorted(risultati, key=lambda x: x[1], reverse=True)[:top_n]

    print(f"\nTitoli simili a '{dati[index]['title']}' (pesi: descrizione {peso_tfidf}, categorie {1 - peso_tfidf}):\n")
    for titolo, score in top:
        print(f"{titolo} — similarità: {round(score, 2)}")




In [46]:
titolo = input("Inserisci il titolo di un film o serie: ")
raccomanda_ibrido(dati, tfidf, titolo, peso_tfidf=0.4)

Inserisci il titolo di un film o serie:  the irishman



Titoli simili a 'The Irishman' (pesi: descrizione 0.4, categorie 0.6):

Gangs of New York — similarità: 0.6
GoodFellas — similarità: 0.55
Working Title — similarità: 0.55
The Shadow of Violence — similarità: 0.55
Cut Throat City — similarità: 0.54


## Bonus track: usiamo gli embedding


In [47]:
!pip install sentence-transformers

  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached torch-2.7.0-cp311-none-macosx_11_0_arm64.whl.metadata (29 kB)
  Using cached scikit_learn-1.6.1-cp311-cp311-macosx_12_0_arm64.whl.metadata (31 kB)
  Using cached pillow-11.2.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (8.9 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached regex-2024.11.6-cp311-cp311-macosx_11_0_arm64.whl.metadata (40 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached safetensors-0.5.3-cp38-abi3-macosx_11_0_arm64.whl.metadata (3.8 kB)
  Using cached fsspec-2025.3.2-py3-none-any.whl.metadata (11 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metada

In [62]:
from sentence_transformers import SentenceTransformer

# modello = SentenceTransformer('all-MiniLM-L6-v2')  # Veloce e buono
modello = SentenceTransformer('all-distilroberta-v1')

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [63]:
def genera_embedding_descrizioni(dati, modello):
    descrizioni = [r['description'] if r['description'] else '' for r in dati]
    return modello.encode(descrizioni, show_progress_bar=True)

In [64]:
from numpy import dot
from numpy.linalg import norm

def similarita_coseno_vec(v1, v2):
    if norm(v1) == 0 or norm(v2) == 0:
        return 0.0
    return dot(v1, v2) / (norm(v1) * norm(v2))


In [65]:
def raccomanda_embedding(dati, embeddings, titolo_input, top_n=5):
    titolo_input = titolo_input.strip().lower()
    index = next((i for i, r in enumerate(dati) if r['title'].strip().lower() == titolo_input), None)

    if index is None:
        print("Titolo non trovato.")
        return

    v_input = embeddings[index]
    risultati = []

    for i, v in enumerate(embeddings):
        if i == index:
            continue
        sim = similarita_coseno_vec(v_input, v)
        risultati.append((dati[i]['title'], sim))

    top = sorted(risultati, key=lambda x: x[1], reverse=True)[:top_n]

    print(f"\nTitoli simili a '{dati[index]['title']}' (con embedding semantico):\n")
    for titolo, score in top:
        print(f"{titolo} — similarità: {round(score, 2)}")


In [66]:
embedding_descrizioni = genera_embedding_descrizioni(dati, modello)


Batches: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 276/276 [00:21<00:00, 12.74it/s]


In [67]:
embedding_descrizioni[0].shape

(768,)

In [68]:

titolo = input("Inserisci il titolo di un film o serie: ")
raccomanda_embedding(dati, embedding_descrizioni, titolo)

Inserisci il titolo di un film o serie:  the irishman



Titoli simili a 'The Irishman' (con embedding semantico):

The Daughter — similarità: 0.5199999809265137
Catch Me If You Can — similarità: 0.49000000953674316
Once Upon a Time in America — similarità: 0.46000000834465027
The Blacklist — similarità: 0.46000000834465027
Papa the Great — similarità: 0.46000000834465027


In [69]:
def raccomanda_embedding_ibrido(dati, embedding_descrizioni, titolo_input, peso_emb=0.7, top_n=5):
    voc_gen = costruisci_vocabolario_generi(dati)
    voc_rat, voc_typ, voc_dir = costruisci_vocabolari_categorici(dati)

    titolo_input = titolo_input.strip().lower()
    index = next((i for i, r in enumerate(dati) if r['title'].strip().lower() == titolo_input), None)

    if index is None:
        print("Titolo non trovato.")
        return

    v_emb_input = embedding_descrizioni[index]
    v_struct_input = vettore_completo(dati[index], voc_gen, voc_rat, voc_typ, voc_dir)

    risultati = []
    for i, r in enumerate(dati):
        if i == index:
            continue

        v_emb = embedding_descrizioni[i]
        v_struct = vettore_completo(r, voc_gen, voc_rat, voc_typ, voc_dir)

        sim_emb = similarita_coseno_vec(v_emb_input, v_emb)
        sim_struct = similarita_coseno(v_struct_input, v_struct)

        sim_finale = peso_emb * sim_emb + (1 - peso_emb) * sim_struct
        risultati.append((r['title'], sim_finale))

    top = sorted(risultati, key=lambda x: x[1], reverse=True)[:top_n]

    print(f"\nTitoli simili a '{dati[index]['title']}' (embedding + colonne):\n")
    for titolo, score in top:
        print(f"{titolo} — similarità: {round(score, 2)}")



In [76]:
titolo = input("Inserisci un titolo: ")
raccomanda_embedding_ibrido(dati, embedding_descrizioni, titolo, peso_emb=0.4)

Inserisci un titolo:  once upon a time in the west



Titoli simili a 'Once Upon a Time in the West' (embedding + colonne):

Quigley Down Under — similarità: 0.6600000262260437
Dances with Wolves — similarità: 0.6399999856948853
Brick Mansions — similarità: 0.6200000047683716
The Outlaw Josey Wales — similarità: 0.6200000047683716
American Outlaws — similarità: 0.6100000143051147


## Ultimo step - Raccomandazioni per profilo utente

In [77]:
def profilo_utente(film_preferiti, dati, embedding_descrizioni):
    film_input = [t.strip().lower() for t in film_preferiti]
    
    indici = [
        i for i, r in enumerate(dati)
        if r['title'].strip().lower() in film_input
    ]

    if not indici:
        print("Nessun titolo trovato.")
        return None

    vettori = [embedding_descrizioni[i] for i in indici]
    vettore_medio = sum(vettori) / len(vettori)
    return vettore_medio

def profilo_utente_ibrido(film_preferiti, dati, embedding_descrizioni, voc_gen, voc_rat, voc_typ, voc_dir):
    film_input = [t.strip().lower() for t in film_preferiti]

    indici = [
        i for i, r in enumerate(dati)
        if r['title'].strip().lower() in film_input
    ]

    if not indici:
        print("Nessun titolo trovato.")
        return None, None

    emb_vettori = [embedding_descrizioni[i] for i in indici]
    struct_vettori = [vettore_completo(dati[i], voc_gen, voc_rat, voc_typ, voc_dir) for i in indici]

    emb_media = sum(emb_vettori) / len(emb_vettori)
    struct_media = [sum(x)/len(struct_vettori) for x in zip(*struct_vettori)]

    return emb_media, struct_media

def raccomanda_ibrido_utente(dati, embedding_descrizioni, film_preferiti, peso_emb=0.7, top_n=5):
    voc_gen = costruisci_vocabolario_generi(dati)
    voc_rat, voc_typ, voc_dir = costruisci_vocabolari_categorici(dati)

    v_emb_user, v_struct_user = profilo_utente_ibrido(film_preferiti, dati, embedding_descrizioni, voc_gen, voc_rat, voc_typ, voc_dir)
    if v_emb_user is None:
        return

    film_input = [t.strip().lower() for t in film_preferiti]
    risultati = []

    for i, record in enumerate(dati):
        titolo = record['title'].strip().lower()
        if titolo in film_input:
            continue

        v_emb = embedding_descrizioni[i]
        v_struct = vettore_completo(record, voc_gen, voc_rat, voc_typ, voc_dir)

        sim_emb = similarita_coseno_vec(v_emb_user, v_emb)
        sim_struct = similarita_coseno(v_struct_user, v_struct)
        sim_finale = peso_emb * sim_emb + (1 - peso_emb) * sim_struct

        risultati.append((record['title'], sim_finale))

    top = sorted(risultati, key=lambda x: x[1], reverse=True)[:top_n]

    print("\n🎯 Raccomandazioni personalizzate (embedding + struttura):\n")
    for titolo, score in top:
        print(f"{titolo} — similarità: {round(score, 2)}")



In [92]:
film_preferiti = [
    "Once upon a time in the west",
    "Dances with Wolves",
    "The Outlaw Josey Wales"
]

raccomanda_ibrido_utente(dati, embedding_descrizioni, film_preferiti, peso_emb=0.4)


🎯 Raccomandazioni personalizzate (embedding + struttura):

Quigley Down Under — similarità: 0.699999988079071
American Outlaws — similarità: 0.6899999976158142
Platoon — similarità: 0.6499999761581421
Cleopatra Jones — similarità: 0.6499999761581421
Wyatt Earp — similarità: 0.6499999761581421
